Elements from the Azure ML sample Jupyter Notebook below are used:  
```Samples\1.43.0\how-to-use-azureml\ml-frameworks\keras\train-hyperparameter-tune-deploy-with-keras\train-hyperparameter-tune-deploy-with-keras.ipynb```

## Set up workspace

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-208063
Azure region: southcentralus
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-208063


## Set up compute cluster

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "StandardD2CPUCluster"

# DW: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive run

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# DW: Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.001, 1), # Inverse of regularization strength
    "--max_iter": choice(25, 50, 75, 100) # Maximum number of iterations to converge
    })

# DW: Specify a Termination Policy (for Early Stopping)
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# DW: Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

# DW: Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [4]:
# DW: Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_4d3d14ce-d308-4539-856c-fadba2a9ce5b
Web View: https://ml.azure.com/runs/HD_4d3d14ce-d308-4539-856c-fadba2a9ce5b?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-208063/workspaces/quick-starts-ws-208063&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2022-09-20T16:49:35.0011603Z][SCHEDULER][INFO]Scheduling job, id='HD_4d3d14ce-d308-4539-856c-fadba2a9ce5b_0' 
[2022-09-20T16:49:34.200834][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-09-20T16:49:35.1566213Z][SCHEDULER][INFO]Scheduling job, id='HD_4d3d14ce-d308-4539-856c-fadba2a9ce5b_1' 
[2022-09-20T16:49:35.2649318Z][SCHEDULER][INFO]Scheduling job, id='HD_4d3d14ce-d308-4539-856c-fadba2a9ce5b_2' 
[2022-09-20T16:49:35.331914][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2022-09-20T16:49:35.6259869Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_4d3d14ce-d308-4539-8

{'runId': 'HD_4d3d14ce-d308-4539-856c-fadba2a9ce5b',
 'target': 'StandardD2CPUCluster',
 'status': 'Completed',
 'startTimeUtc': '2022-09-20T16:49:33.500864Z',
 'endTimeUtc': '2022-09-20T17:09:43.421292Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'e9188092-d9e4-42b8-8564-2be1b9d92d24',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1017-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.44.0',
  'space_size': 'infinite_space_size',
  'score': '0.9099295945617868',
  'best_child_run_id': 'HD_4d3d14ce-d308-4539-856c-fadba2a9ce5b_7',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_4d3d14ce-d308-4539-856c-fadba2a9ce5b_7'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'tele

In [5]:
import joblib
# Get your best run and save the model from that run.

# DW: First, get the best run
best_run = hyperdrive_run.get_best_run_by_primary_metric() # The best run based on the primary metric
print(best_run.id) # Display the ID of the best run
print(best_run.get_metrics()) # Display the parameters and primary metric values of the best run

HD_4d3d14ce-d308-4539-856c-fadba2a9ce5b_7
{'Regularization Strength:': 0.5998557382016532, 'Max iterations:': 75, 'Accuracy': 0.9099295945617868}


In [6]:
best_run.get_file_names() # DW: Next, identify the file names for the best run

['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2022-09-20-17',
 'outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [7]:
# DW: Finally, register the model of the best run to save it
hd_model = best_run.register_model(model_name = 'hd_model',
                                   model_path = 'outputs/model.joblib')

## AutoML run

In [48]:
from azureml.data.dataset_factory import TabularDatasetFactory

# DW: Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

dsurl = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(dsurl)

In [49]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [50]:
# DW: Let's first take a look at the data.
x.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_cellular,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown
0,57,1,0,0,1,5,1,371,1,999,...,1,0,0,0,0,1,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,0,1,0,0,0,0,0,0,0,1
2,33,1,0,0,0,5,5,52,1,999,...,1,0,0,0,1,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,0,1,0,0,0,1,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,1,0,0,0,0,1,0,0,0,0


In [51]:
# And now see what the output results look like.
y.value_counts()

0    29258
1     3692
Name: y, dtype: int64

So we have that many numerical field are present, and we want to use them to predict a 0 or 1 output. As such, this is a classification problem.

In [52]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='Accuracy',
    training_data=x,
    label_column_name=y,
    n_cross_validations=5)

In [53]:
# DW: Submit your automl run

automl_run = exp.submit(automl_config, show_output=True)
automl_run.wait_for_completion(show_output=True)

ConfigException: ConfigException:
	Message: Argument [label_column_name] is of unsupported type: [<class 'pandas.core.series.Series'>]. Supported type(s): [int, str]
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Argument [label_column_name] is of unsupported type: [<class 'pandas.core.series.Series'>]. Supported type(s): [int, str]",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "label_column_name",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid"
            }
        },
        "reference_code": "061ed905-e59e-42b9-ad95-cd18f40b4358"
    }
}

In [ ]:
# DW: Retrieve and save your best automl model.

automl_best_run = automl_run.get_best_run_by_primary_metric() # The best run based on the primary metric
print(automl_best_run.get_metrics()) # Display the parameters and the primary metric values of the best run

In [54]:
# Finally, delete the CPU cluster to tidy up the resources.
cpu_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

